In [31]:

import json
import numpy as np
from son_main_script import Son
from son_pymoo import HighRssiFirstSampling, SonProblemElementWise
from pymoo.decomposition.asf import ASF
import plotly.graph_objects as go
from pymoo.indicators.hv import Hypervolume
import matplotlib.pyplot as plt
import scipy.stats
from pltyCon.plotly_templates import load_templates
from PyPDF2 import PdfWriter, PdfReader 
import glob
import os

template_one_diagram = "oscilloscope+TH--2-3-TH"
greedy_color= "black"
evo_color= "blue"
lineWidth = 1

loaded_dpi = load_templates(
    # base_width_in_px=400,
    dpi=96
)

def asf_select(objective_space: np.ndarray,
                    weights=[]):

    approx_ideal = objective_space.min(axis=0)
    approx_nadir = objective_space.max(axis=0)

    np.seterr(divide='ignore', invalid='ignore')
    nF = (objective_space - approx_ideal) / (approx_nadir - approx_ideal)
    decomp = ASF()

    if len(weights) != objective_space.shape[1]:
        weights = [1/objective_space.shape[1] for _ in range(objective_space.shape[1])]

    weights_np= np.array(weights)
    i = decomp.do(nF, weights_np).argmin()
    return i

def normalize_objective_space_pop(objective_space_pop: np.ndarray, ideal, nadir):
    return (objective_space_pop - ideal) / (nadir - ideal)

def create_greedy_rssi_sampling_pop(optimization_objectives: list[str], son: Son):
     ###### test HighRssiFirstSampling
    
    son_problem_obj = SonProblemElementWise(optimization_objectives, son)
    # create pop with RssiSampling
    high_rssi_sampling_obj = HighRssiFirstSampling()
    sampling_pop = high_rssi_sampling_obj._do(son_problem_obj, 100)
    sampling_pop_objecitve_space = np.empty((100,len(optimization_objectives)))
    out = {}
    for index, ind in enumerate(sampling_pop):
        son_problem_obj._evaluate(ind, out)
        sampling_pop_objecitve_space[index]=out["F"]
    return sampling_pop_objecitve_space

def get_hypervolume(objective_space_pop: np.ndarray, ref_point=np.array([1.1, 1.1]), norm_ref_point=False, zero_to_one=False, ideal=None, nadir=None):
    hv = Hypervolume(ref_point=ref_point,
                     norm_ref_point=norm_ref_point,
                     zero_to_one=zero_to_one,
                     ideal=ideal,
                     nadir=nadir)
    
    return hv(objective_space_pop)
def remove_blank_page(directory_path:str):

    pdf_files = glob.glob(os.path.join(directory_path, '*.pdf'))
    for pdf_file in pdf_files:
        infile = PdfReader(pdf_file)

        outfile = PdfWriter()
        pdfPage = infile.pages[0]
        outfile.add_page(pdfPage)

        with open(pdf_file, "wb") as f:
            outfile.write(f) 

##################################################################



# network_name = "het_C50_MP100_1,2_250"
network_name = "het_C50_MP30_1,2_250"
config_names = [ "static"]



##################################################################
pareto_fronts = []
paretor_fronts_normalized = []
hyper_volume_histories = []
combined_his = [] ## -> combine all objective result in one array (used only for normalization)
nadir = np.array([])
ideal = np.array([])
son = Son(adjacencies_file_name=f"./datastore/{network_name}/{network_name}_adjacencies.json", parameter_config_file_name=f"./datastore/{network_name}/{config_names[0]}/{config_names[0]}.json")


### find  nadir and ideal points for normalization over all experiments
for _, config_name in enumerate(config_names):
    results_directory = f"./datastore/{network_name}/{config_name}/"

    with open(results_directory + "objectives_result.json", mode="r",encoding="utf-8") as openfile:
        results_object = json.load(openfile)

        for _, current_pop_objectivespace in enumerate(results_object["history"]["objective_space_opt"]):
            for _, current_ind in enumerate(current_pop_objectivespace):
                combined_his.append(list(np.array(current_ind)[0:2]))
        
        # jo = np.array(results_object["objectiveSpace"])[:,0:2]
        # for _, current_ind in enumerate(jo):
        #     combined_his.append(list(current_ind))

sampling_pop_objecitve_space = create_greedy_rssi_sampling_pop(results_object["optimization_objectives"], son)
# sampling_pop_objecitve_space = sampling_pop_objecitve_space[-1::]

for _, current_ind in enumerate(sampling_pop_objecitve_space):
    combined_his.append(list(current_ind))


nadir = np.array(combined_his).max(axis=0)
ideal = np.array(combined_his).min(axis=0)

sampling_pop_objecitve_space_noramlized = normalize_objective_space_pop(sampling_pop_objecitve_space, ideal, nadir)

### collect paretorfronts, hypervolumes, normalized paretorfronts per experiment
for _, config_name in enumerate(config_names):
    results_directory = f"./datastore/{network_name}/{config_name}/"

    
    with open(results_directory + "objectives_result.json", mode="r",encoding="utf-8") as openfile:
        results_object = json.load(openfile)

        pareto_fronts.append(np.array(results_object["objectiveSpace"])[:,0:2])
        paretor_fronts_normalized.append(normalize_objective_space_pop(np.array(results_object["objectiveSpace"])[:,0:2], ideal, nadir))
        hyper_volume_history = []
            
        for _, current_pop_objectivespace in enumerate(results_object["history"]["objective_space_opt"]):
            normalized_current_pop = normalize_objective_space_pop(np.array(current_pop_objectivespace)[:,0:2], ideal, nadir)
            hyper_volume_history.append(get_hypervolume(np.array(normalized_current_pop)))
        
    hyper_volume_histories.append(hyper_volume_history)

In [32]:

fig0 = go.Figure()
fig0.update_layout(
    xaxis_title="PC",
    yaxis_title="-AVG_DL",
    template=template_one_diagram,
    showlegend=False
    )
asf_selection_point = (1,2)

##### add paretofronts of all configs
for index, pareto_front in enumerate(pareto_fronts):

    fig0.add_trace(
        go.Scatter(
            x=np.array(paretor_fronts_normalized[index])[:,0],
            y=np.array(paretor_fronts_normalized[index])[:,1],
            mode='markers',
            line={"color": evo_color},
            name=config_names[index],
            )
        )
    
    i_2 = asf_select(np.array(pareto_front))
  
    fig0.add_trace(
    go.Scatter(
        x=np.array(paretor_fronts_normalized[index][i_2][0]),
        y=np.array(paretor_fronts_normalized[index][i_2][1]),
        mode='markers',
        name= "asf selection",
        # opacity= 0.5,
        # marker=dict(size=10)
        )
    )
    asf_selection_point= ( paretor_fronts_normalized[index][i_2][0], paretor_fronts_normalized[index][i_2][1])

##### add rssi sampling population

fig0.add_trace(
     go.Scatter(
        x=sampling_pop_objecitve_space_noramlized[:,0],
        y=sampling_pop_objecitve_space_noramlized[:,1],
        mode='markers',
        # line={"color": greedy_color},
        name= "greedy_sampling_pop",
        )
    )
###### highlight greedy assignment activation profile

fig0.add_trace(
    go.Scatter(
        x=np.array(sampling_pop_objecitve_space_noramlized[-1][0]),
        y=np.array(sampling_pop_objecitve_space_noramlized[-1][1]),
        mode='markers',
        name= "greedy_assignment_profile",
        line={"color": greedy_color},
        )
    )
i = asf_select(sampling_pop_objecitve_space)

# fig0.add_trace(
#     go.Scatter(
#         x=[sampling_pop_objecitve_space_noramlized[i][0],sampling_pop_objecitve_space_noramlized[i][0], asf_selection_point[0], asf_selection_point[0]],
#         y=[sampling_pop_objecitve_space_noramlized[i][1], asf_selection_point[1], asf_selection_point[1], sampling_pop_objecitve_space_noramlized[i][1]],
#         fill="toself",
#         mode="lines+text",
#         line_width=0,
#     )
# )

# volume = (asf_selection_point[0]-sampling_pop_objecitve_space_noramlized[i][0])*(asf_selection_point[1]-sampling_pop_objecitve_space_noramlized[i][1])

# fig0.add_annotation(
#     x=0.6, y=0.6, text=round(volume,4),
#     showarrow=False,
#     font=dict(
#             size=16,
#             ),
#     )

fig2 = go.Figure()
fig2.update_layout(
    title="hypervolume over n_gen",
    xaxis_title="generation",
    yaxis_title="hypervolume")

for index, hyper_volume_history in enumerate(hyper_volume_histories):
    fig2.add_trace(
        go.Scatter(
            x=list(range(0, len(hyper_volume_history))),
            y=hyper_volume_history,
            mode='lines',
            name=config_names[index],
            ))


# fig1.show()
fig0.show()
# fig0.write_image(f"./diagrams/paretofront_{network_name}.pdf")
fig0.write_image(f"./diagrams/{network_name}_static_opt_example.pdf")
remove_blank_page("./diagrams/")
fig2.show()

In [33]:

cor_matrix = False

def print_cor_matrix():
        network_name = "het_C100"
        config_name = "static_evo_var"

        results_directory = f"./datastore/{network_name}/{config_name}/"

        with open(results_directory + "objectives_result.json", mode="r",encoding="utf-8") as openfile:
                results_object = json.load(openfile)
                paretofront = np.array(results_object["objectiveSpace"])

                nadir = paretofront.max(axis=0)
                ideal = paretofront.min(axis=0)
                paretofront = np.array((paretofront - ideal) / (nadir - ideal))


                xyz = paretofront.transpose()

                corr_matrix, p_matrix = scipy.stats.spearmanr(xyz, axis=1)
                corr_matrix = np.array(corr_matrix)
                corr_matrix = corr_matrix.round(4)

                print(corr_matrix)
                fig, ax = plt.subplots()
                im = ax.imshow(corr_matrix)
                im.set_clim(-1, 1)
                ax.grid(False)
                ax.xaxis.set(ticks=(0, 1, 2), ticklabels=('PWR_CONSUMPTION', 'AVG_DL', 'DL_VARIANCE'))
                ax.yaxis.set(ticks=(0, 1, 2), ticklabels=('PWR_CONSUMPTION', 'AVG_DL', 'DL_VARIANCE'))
                ax.set_ylim(2.5, -0.5)
                for i in range(3):
                        for j in range(3):
                                ax.text(j, i, corr_matrix[i, j], ha='center', va='center',
                                        color='r')
                cbar = ax.figure.colorbar(im, ax=ax, format='% .2f')
                plt.show() 
if cor_matrix:
        print_cor_matrix()

